# Virological data – Fetch, load, visualize and test

In [1]:
import os
import sys

import epiweeks
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px

sys.path.append("../../../Rtrend_project/")  # MANUAL: Must be done in a different way in the future

if os.path.basename(os.getcwd()) != "programs":
    os.chdir("..")  # Move to project root
print(f"Working directory: {os.getcwd()}")


# ============================= Tools – To be moved to other modules

def get_epiweek_and_year(date: pd.Timestamp):
    week_obj = epiweeks.Week.fromdate(date.date())
    return week_obj.week, week_obj.year


def get_fluview_season(date: pd.Timestamp, fluview_first_week=40) -> str:
    """
    Given a date, determine the FluView season to which it belongs.
    
    Parameters
    ----------
    date : pd.Timestamp
        The date to check.
    fluview_first_week : int
        Epiweek that delimits the first week of the FluView season. 
        In the FluView database, every season starts at the 40th epiweek
        of the current year (~October). 

    Returns
    -------
    str
        The FluView season as a string, e.g. '2022-2023'.
    """
    date = pd.Timestamp(date)  # Ensure type, convert if needed
    # Check if date is within range of available data
    if date < pd.to_datetime('2016-10-08') or date > pd.Timestamp.now():
        raise ValueError(f"Date {date.date()} is outside of available FluView data range.")
    
    # Find the epi week index and year of that date
    # epiweek, _, iso_year = date.isocalendar()
    week_obj = epiweeks.Week.fromdate(date.date())
    epi_week = week_obj.week
    epi_year = week_obj.year
    
    # Determine the season
    if epi_week >= fluview_first_week:
        return f"{epi_year}-{epi_year+1}"
    else:
        return f"{epi_year-1}-{epi_year}"


# def get_fluview_required_seasons_and_weeks()

Working directory: /Users/pventura/NonCloudStorage/Flu_forecast/programs


## Fetching and preprocessing

Virological data source: CDC FluView reports – https://www.cdc.gov/fluview/index.html 

In [2]:
# URL template. Fed by `season_str` (eg. "2024-2025") and `epiweek` (as integer). The season year is the first on 
url_fmt = "https://www.cdc.gov/flu/weekly/weeklyarchives{season_str}/data/whoAllregt_phl{epiweek:02d}.html"

# Set the range of interest for fetching virological data
# viro_roi_start, viro_roi_end = pd.Timestamp("2017-01-01"), pd.Timestamp("2024-10-01")
viro_roi_start, viro_roi_end = pd.Timestamp("2017-01-01"), pd.Timestamp("2025-01-28")  # The given URL stops working at https://www.cdc.gov/flu/weekly/weeklyarchives2023-2024/data/whoAllregt_phl52.html




# first_season, last_season = [get_fluview_season(date) for date in [viro_roi_start, viro_roi_end]]


# ======================= CALCULATE what data must be requested from FluView API. This can become a separate toolset.
# Here, I need a toolset that can identify required *seasons* based on a date range.
# - Define the limits of each season (is it consistent over time in CDC?)
# - Determine the latest file of each season to send them to the fetch procedure.

# Get the limiting seasons (string: "{y}-{y+1}"), then extract their year as integers (y) 
first_season, last_season = [get_fluview_season(date) for date in [viro_roi_start, viro_roi_end]]
first_year, last_year = [int(s.split("-")[0]) for s in [first_season, last_season]]

# Create the object with data for the API requests
# Interpolate, setting the last week as 39 for all years
request_data_dict = {
    year: {"season_str": f"{year}-{year+1}", "epiweek": 39} 
    for year in range(first_year, last_year + 1)  
}

# -()- Set the week of the last year to the last week required
request_data_dict[last_year]["epiweek"] = get_epiweek_and_year(viro_roi_end)[0]
# -()- MANUALLY CHANGE, since this logic didn't hold in January
# request_data_dict[last_year]["epiweek"] = 52
# request_data_dict[last_year]["season_str"] = "2024-2025"

print(request_data_dict)


# =================
# Load the data
df_list = list()

for yr, request_data in request_data_dict.items():
    url = url_fmt.format(**request_data)
    print(f"Requesting {url}")
    fluview_df = pd.read_html(url)[0]

    df_list.append(fluview_df)

fluview_df = pd.concat(df_list, axis=0, ignore_index=True)

# =======================

# BACKUP THE DATA
# ///////////////
fname = f"aux_data/virological/fluview_{viro_roi_start.date().isoformat()}_to_{viro_roi_end.date().isoformat()}.csv"
print(f"Saving to file: {fname}")
fluview_df.to_csv(fname, index=False)

# # -------------
# url = url_fmt.format(season_str="2023-2024", epiweek=20)
# fluview_df = pd.read_html(url)[0]
# 
# fluview_df

{2016: {'season_str': '2016-2017', 'epiweek': 39}, 2017: {'season_str': '2017-2018', 'epiweek': 39}, 2018: {'season_str': '2018-2019', 'epiweek': 39}, 2019: {'season_str': '2019-2020', 'epiweek': 39}, 2020: {'season_str': '2020-2021', 'epiweek': 39}, 2021: {'season_str': '2021-2022', 'epiweek': 39}, 2022: {'season_str': '2022-2023', 'epiweek': 39}, 2023: {'season_str': '2023-2024', 'epiweek': 39}, 2024: {'season_str': '2024-2025', 'epiweek': 5}}
Requesting https://www.cdc.gov/flu/weekly/weeklyarchives2016-2017/data/whoAllregt_phl39.html
Requesting https://www.cdc.gov/flu/weekly/weeklyarchives2017-2018/data/whoAllregt_phl39.html
Requesting https://www.cdc.gov/flu/weekly/weeklyarchives2018-2019/data/whoAllregt_phl39.html
Requesting https://www.cdc.gov/flu/weekly/weeklyarchives2019-2020/data/whoAllregt_phl39.html
Requesting https://www.cdc.gov/flu/weekly/weeklyarchives2020-2021/data/whoAllregt_phl39.html
Requesting https://www.cdc.gov/flu/weekly/weeklyarchives2021-2022/data/whoAllregt_phl

In [3]:
fluview_df["Week"].max()

202505

In [4]:
# Preprocessing
# ===============

map_fluview_names = {
    "Total # Tested": "total_tested",
    "A(H3)": "A_h3",
    "A (H1N1)pdm09": "A_h1n1_pdm09",
    "A(H3N2v)": "A_h3n2v",
    "A(unable to sub-type)": "A_unable",
    "A(Subtyping not performed)": "A_not_performed",
    # "B": "B",
    "BVIC": "B_victoria",
    "BYAM": "B_yamagata",
}

viro_cols = ["A_h3", "A_h1n1_pdm09", "A_h3n2v", "B_victoria", "B_yamagata"]

# ------------
df = fluview_df.copy()
df = df.rename(mapper=map_fluview_names, axis=1)

# --- Map week integers to pandas dates
def epiweek_nr_to_date(n):
    """`n` is expected as an integer like: 202412 – Year = 2024, week = 12"""
    year = n // 100
    week = n % 100
    return pd.Timestamp(epiweeks.Week(year, week).enddate())

df["week_end_date"] = df["Week"].map(epiweek_nr_to_date) 

# # --- Checkpoint – Visualize the data
# fig = px.bar(df, x="week_end_date", y=viro_cols)
# fig.update_layout(title_text="Influenza positive tests by subtype")
# fig.show()

In [5]:
# with open("tmp_figs/fluview_data.html", "w") as fp:
#     fp.write(fig.to_html())

In [6]:
# Plotly is misbehaving... let's do all plotly part separately
# ================================
print("Making plot...")
fig = px.bar(df, x="week_end_date", y=viro_cols)
print("Adding text")
fig.update_layout(title_text="Influenza positive tests by subtype")
print("Showing fig")
fig.show()

print("Saving fig")
with open("tmp_figs/fluview_data.html", "w") as fp:
    fp.write(fig.to_html())

Making plot...
Adding text
Showing fig


Saving fig
